# Triggering observation of ANTARES locus objects with TOM

<img src="tom.jpeg">

**This notebook requires the installation of ANTARES client (https://noao.gitlab.io/antares/client/) and TOMtoolkit (https://tom-toolkit.readthedocs.io/en/latest/introduction/getting_started.html#installing-the-tom-toolkit-and-django). For more detail on programmatic access of TOMtoolkit, please see (https://tom-toolkit.readthedocs.io/en/stable/common/scripts.html).**



We can arrange follow up observations of intriguing ANTARES locus/alert using the facilities within the Astronomical Event Observatory Network (AEON). This can be conviently done with the TOMtoolkit as follows.

The first step is to define the target information (name, ra, dec, etc.)

In [69]:
from antares_client.search import get_by_id, get_by_ztf_object_id
#get locus by ANTARES ID
locus = get_by_id("ANT2018c7igm")

#get locus by ZTF ID
#locus = get_by_ztf_id("ZTF18abhjrcf")

print(locus.locus_id, locus.ra, locus.dec)

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from tom_targets.models import Target
t = Target.objects.create(name=locus.locus_id, type='SIDEREAL', ra=locus.ra, dec=locus.dec)

ANT2018c7igm 280.6927190683333 -12.904117143333334


We can see there is now a new target added

<img src="tom_target.jpeg">

The next step is to populate the observation form

In [64]:
from tom_observations.facilities.lco import LCOFacility, LCOBaseObservationForm

target = Target.objects.get(name=locus.locus_id)

form = LCOBaseObservationForm({
    'name': 'Programmatic Observation',
    'proposal': 'TOM2020A-012',
    'ipp_value': 1.05,
    'start': '2020-10-09T00:00:00',
    'end': '2020-10-10T00:00:00',
    'filter': 'R',
    'instrument_type': '1M0-SCICAM-SINISTRO',
    'exposure_count': 1,
    'exposure_time': 20,
    'max_airmass': 4.0,
    'observation_mode': 'RAPID_RESPONSE',
    'target_id': target.id,
    'facility': 'LCO'
})

We can check if there is any error of the observation form using form.is_valid()

In [65]:
form.is_valid()

True

Once the form is validated, we can submit it using the following command:

In [66]:
observation_ids = LCOFacility().submit_observation(form.observation_payload())
print(observation_ids)

[2259759]


We can also create a record of the observation request

In [67]:
from tom_observations.models import ObservationRecord
for observation_id in observation_ids:
    record = ObservationRecord.objects.create(
        target=target,
        facility='LCO',
        parameters=form.serialize_parameters(),
        observation_id=observation_id
    )
    print(record)

Observation change state hook: ANT2018c7igm @ LCO from None to 


ANT2018c7igm @ LCO


Now we can see a pending observation in TOM

<img src="tom_obsrecord.jpeg">

We can also see the observation request at the LCO observation portal:
    
<img src="lco_request.jpeg">    